# Week 7 — Part 04: Testing Strategy Lab

**Estimated time:** 90–120 minutes

---

## Pre-study (Self-learn)

Foundamental Course assumes Self-learn is complete. If you need a refresher on testing and project structure:

- [Foundamental Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Chapter 2: Python and Environment Management](../self_learn/Chapters/2/Chapter2.md)

---

## What success looks like (end of Part 04)

- You can write pytest tests for your pipeline.
- You can create smoke tests for integration testing.
- You can test happy paths, edge cases, and failure cases.

### Checkpoint

After running this notebook:
- You have pytest tests written
- You can run tests with `pytest`

## Learning Objectives

- Write pytest unit tests
- Create smoke tests for pipelines
- Test different scenarios (happy path, edge cases, failures)

## Overview

Goal: add a small set of checks that protect you from regressions.

In this lab you will:

- define a minimal 3+ case plan (happy/edge/failure)
- write checks that assert contracts (JSON valid, required keys, artifacts exist)
- choose pytest unit tests or a smoke-test script

If you want the deeper theory on why tests reduce uncertainty, use the Self-learn links at the top of the notebook.

## Minimal test plan (3+ cases)

You should have at least:

- **happy path**: normal input works
- **edge case**: missing values or tiny CSV
- **failure case**: missing file / invalid schema

Practical tip: smoke tests can be “one command” checks that run in CI later. The key is making them deterministic enough to be repeatable.

import json
import tempfile
from pathlib import Path


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError("Input file not found: %s" % p)
    if p.stat().st_size == 0:
        raise ValueError("Input file is empty: %s" % p)
    return p


def test_require_file_missing(tmp_path):
    # TODO: implement pytest unit test for missing file.
    # Example: call require_file() and assert FileNotFoundError.
    missing = Path(tmp_path) / "missing.csv"
    try:
        _ = require_file(str(missing))
    except FileNotFoundError:
        return
    raise AssertionError("expected FileNotFoundError")


def test_require_file_empty(tmp_path):
    # TODO: implement pytest unit test for empty file.
    p = Path(tmp_path) / "empty.csv"
    p.write_text("", encoding="utf-8")
    try:
        _ = require_file(str(p))
    except ValueError:
        return
    raise AssertionError("expected ValueError")


def smoke_test_pipeline() -> None:
    # TODO: run an end-to-end pipeline with a tiny input.
    # Verify artifacts exist: output/report.json, output/report.md
    out_dir = Path("output")
    out_dir.mkdir(exist_ok=True)
    (out_dir / "report.json").write_text(json.dumps({"ok": True}, indent=2), encoding="utf-8")
    (out_dir / "report.md").write_text("# Report\n\nOK", encoding="utf-8")


with tempfile.TemporaryDirectory() as d:
    test_require_file_missing(d)
    test_require_file_empty(d)

smoke_test_pipeline()
print("wrote:", Path("output") / "report.json")
print("Implement pytest tests + smoke_test_pipeline().")

## References

- pytest docs: https://docs.pytest.org/

## Appendix: Solutions (peek only after trying)

Reference implementations for pytest-style tests and a smoke test.

In [ ]:
def test_require_file_missing(tmp_path):
    import pytest

    missing = Path(tmp_path) / "missing.csv"
    with pytest.raises(FileNotFoundError):
        _ = require_file(str(missing))


def test_require_file_empty(tmp_path):
    import pytest

    p = Path(tmp_path) / "empty.csv"
    p.write_text("", encoding="utf-8")
    with pytest.raises(ValueError):
        _ = require_file(str(p))


def smoke_test_pipeline() -> None:
    out_dir = Path("output")
    out_dir.mkdir(exist_ok=True)
    (out_dir / "report.json").write_text(json.dumps({"ok": True}, indent=2), encoding="utf-8")
    (out_dir / "report.md").write_text("# Report\n\nOK", encoding="utf-8")


print("solution smoke wrote:", Path("output") / "report.json")